In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("IJCDS040403.pdf")
docs = loader.load()
docs

[Document(metadata={'source': 'IJCDS040403.pdf', 'page': 0}, page_content=' \n \n \nInternational Journal of Computing and Digital Systems  \nISSN (2210 -142X)  \nInt. J. Com. Dig. Sys. 4, No.4  (Oct-2015) \n \n \nE-mail: Karthik.Sivakoti@student.csulb.edu , Mohammad.Mozumdar @csulb.edu  \n  http://journals.uob.edu.bh   \nLoad Prediction in Smart Grid Networks  \n \nKarthik Sivakoti and Moham mad Mozumdar  \n  \n Electrical  Engineering, California State University Long Beach, USA  \n \nReceived 6 June  2015, Revised 13 July 2015, Accepted 31 August  2015, Published 1 October  2015 \n \nAbstract: Efficient forecasting  and load prediction for maintain ing the accu rate DR (Demand Response) ratio is a key factor in \nimplementing and deploying the Smart -Grid networks [1]. There are a plethora of techniques and models suggested by forecasters \nover the decades, the most accurate and feasible being – artificial n eural networks, linear regression t echnique and the curve f itting \nalgo

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

[Document(metadata={'source': 'IJCDS040403.pdf', 'page': 0}, page_content='International Journal of Computing and Digital Systems  \nISSN (2210 -142X)  \nInt. J. Com. Dig. Sys. 4, No.4  (Oct-2015) \n \n \nE-mail: Karthik.Sivakoti@student.csulb.edu , Mohammad.Mozumdar @csulb.edu  \n  http://journals.uob.edu.bh   \nLoad Prediction in Smart Grid Networks  \n \nKarthik Sivakoti and Moham mad Mozumdar  \n  \n Electrical  Engineering, California State University Long Beach, USA  \n \nReceived 6 June  2015, Revised 13 July 2015, Accepted 31 August  2015, Published 1 October  2015 \n \nAbstract: Efficient forecasting  and load prediction for maintain ing the accu rate DR (Demand Response) ratio is a key factor in \nimplementing and deploying the Smart -Grid networks [1]. There are a plethora of techniques and models suggested by forecasters \nover the decades, the most accurate and feasible being – artificial n eural networks, linear regression t echnique and the curve f itting'),
 Document(me

In [4]:
documents=text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'IJCDS040403.pdf', 'page': 0}, page_content='International Journal of Computing and Digital Systems  \nISSN (2210 -142X)  \nInt. J. Com. Dig. Sys. 4, No.4  (Oct-2015) \n \n \nE-mail: Karthik.Sivakoti@student.csulb.edu , Mohammad.Mozumdar @csulb.edu  \n  http://journals.uob.edu.bh   \nLoad Prediction in Smart Grid Networks  \n \nKarthik Sivakoti and Moham mad Mozumdar  \n  \n Electrical  Engineering, California State University Long Beach, USA  \n \nReceived 6 June  2015, Revised 13 July 2015, Accepted 31 August  2015, Published 1 October  2015 \n \nAbstract: Efficient forecasting  and load prediction for maintain ing the accu rate DR (Demand Response) ratio is a key factor in \nimplementing and deploying the Smart -Grid networks [1]. There are a plethora of techniques and models suggested by forecasters \nover the decades, the most accurate and feasible being – artificial n eural networks, linear regression t echnique and the curve f itting'),
 Document(me

In [5]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(documents[:30],OpenAIEmbeddings())

c:\Users\sivak\OneDrive\Documents\GitHub\Langchain\langchain_environment\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [6]:
db

In [7]:
query="Why do we need load prediction in smart grids?"
result=db.similarity_search(query)
result[0].page_content

'networks have started to replace the conventional electric \nutilities in recent times, owing to it being an optimized \nself-healing, intelligent monitoring, communication and \ncontrol technology. Apart from being more eco -friendly \nand sustainable, it could provide a higher resilience, \nreliability and support for the ever -growing economy.   \nAccurate forecasting o f the demand in power \nload is one of the pivotal aspects, which  needs to be \nconsidered while focusing on the framework of the Smart -\nGrid network. Predicting the consumed load accurately \nlimits the wastage of energy and saves astronomical \nfigures of dollars  invested by the power utilities on a \nglobal scale. To add to the plethora of benefits derived, it would even greatly reduce the greenhouse emissions \nowing to the matching production and consumption ratio. \nForecasters have come up with various algorithms a iming \nat curbing the above mentioned challenges of which –'

In [8]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm

Ollama()

In [9]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [10]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [11]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024C6498D190>)

In [12]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [14]:
response=retrieval_chain.invoke({"input":"Neural Networks"})

In [15]:
response['answer']

'Based on the provided context, the answer to the question "What are neural networks?" is:\n\nArtificial neural networks (ANNs) are computational models inspired by the structure and function of the human brain. They are composed of interconnected nodes or neurons that process information and learn from experience through a series of interactions. ANNs can be trained to perform various tasks, such as pattern recognition, classification, and prediction. In the context of power systems, ANNs have been used for load forecasting, peak load forecasting, and short-term load forecasting. The Levenberg-Marquardt algorithm, Gauss-Newton algorithm, and genetic algorithms are some of the methods used to train ANNs.\n\nIn addition to the traditional feedforward neural network algorithm, other techniques such as linear regression algorithm and curve fitting have also been found to be accurate in power system forecasting. However, these techniques do not provide the same level of flexibility and ada